In [1]:
import pandas as pd
import os

import os, csv
from tqdm import tqdm
from PIL import Image
from datetime import datetime

In [ ]:
def read_grey(png_path):
    image = Image.open(png_path)
    width, height = image.size
    size = width*height
    r_values = []
    for y in range(height):
        for x in range(width):
            rgb_tuple = image.getpixel((x, y))
            r_values.append(rgb_tuple)

    # 创建一个空字典来存储数字和对应的次数
    count_dict = {}

    # 遍历数字列表
    for num in r_values:
        # 如果数字已经在字典中，增加对应的次数
        if num in count_dict:
            count_dict[num] += 1/size
        # 如果数字不在字典中，将其添加到字典并设置次数为1
        else:
            count_dict[num] = 1/size
    return count_dict

In [ ]:
def extract_ss_from_grey(image_path,image_name,ss_grey_retain_csv, ss_grey_reject_csv):
    # 创建空的列表记录结果
    rate_list = [image_name]
    ss_grey_rate = [0] * 150
    rate_list.extend(ss_grey_rate)

    r_count_dict = read_grey(image_path)
    for num, count in r_count_dict.items():
        rate_list[num+1]=count

    # 剔除元素
    # 3-天空
    # 4-地板；地面
    # 5-树 
    # 6-天花板
    # 7-道路、路线
    # 10-草
    # 12-人行道
    # 14-地球；土地
    # 16-桌子
    # 18-植物；植被；植物界
    # 21-车
    # 22-水
    # 27-海
    # 30-田野
    # 35-岩石；石头
    # 47-沙滩、沙子
    # 53-小路
    # 77-船
    # 81-游大巴；双层巴士；公共小巴；壁托巴士；豪华大巴；公共汽车；载客车辆
    # 84-卡车;货车
    # 92-泥地赛道
    # 103-货车;面包车
    ss_classes = [3,4,5,6,7,10,12,14,16,18,21,22,27,30,35,47,53,77,81,84,92,103]
    ss_classes = [2]
    ratio_threshold = 0.35 # 数据清洗参考的占比值

    # 将结果写入csv
    ss_filter=0
    
    for i in ss_classes:
        ss_filter += rate_list[i]

    # 保留非建筑元素小于0.4的瓦片
    if ss_filter > ratio_threshold:
        with open('%s' % ss_grey_retain_csv ,'a' ,newline='') as f:
            writer = csv.writer(f)
            writer.writerow([image_name])
    else:
        with open('%s' % ss_grey_reject_csv ,'a' ,newline='') as f:
            writer = csv.writer(f)
            writer.writerow([image_name])

In [ ]:
image_folder= r'E:\work\spatio_evo_urbanvisenv_svi\sv\sv_grey'
ss_grey_retain_csv = os.path.join(r'E:\work\spatio_evo_urbanvisenv_svi\sv',"ss_degree_grey_retain.csv") #留下
ss_grey_reject_csv = os.path.join(r'E:\work\spatio_evo_urbanvisenv_svi\sv',"ss_degree_grey_reject.csv") #去除

# 设置表头
headers = ['id',]
# pa_data = pd.read_csv('./data/object150_info.csv', encoding='utf8')
# name_series = pa_data['Name']

# 创建csv文件及表头
# for head in name_series:
#     headers.append(head)

with open('%s'%ss_grey_retain_csv ,'w' ,newline='') as f: 
    writer = csv.writer(f)
    writer.writerow(headers)

with open('%s'%ss_grey_reject_csv ,'w' ,newline='') as f: 
    writer = csv.writer(f)
    writer.writerow(headers)

roots = []
img_names = []
img_paths = []

accepted_formats = (".png", ".jpg", ".JPG", ".jpeg")

for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file.endswith(accepted_formats):
            roots.append(root)
            img_names.append(file)
            file_path = os.path.join(root, file)
            img_paths.append(file_path)

for i, image_name in tqdm(enumerate(img_names)):
    image_path = img_paths[i]
    extract_ss_from_grey(image_path,image_name,ss_grey_retain_csv, ss_grey_reject_csv)

24304it [5:54:19,  1.23s/it]